In [1]:
# Importation des modules utilisés
import sqlite3
import pandas
import pymongo

# Création de la connexion
conn = sqlite3.connect("ClassicModel.sqlite")

In [ ]:
# Récupération du contenu de Customers avec une requête SQL
customers = pandas.read_sql_query("SELECT * FROM Customers;", conn)
print(customers)

Employee = pandas.read_sql_query("SELECT * FROM Employees;", conn)


Q1= pandas.read_sql_query("""SELECT t1.*
            from Customers as t1
            left join Orders as t2 on t1.customerNumber=t2.customerNumber
            where t2.orderNumber is NULL""",conn)
print(Q1)


Q2= pandas.read_sql_query("""SELECT t1.lastName, t1.firstName,
    Count(distinct t2.customerNumber) as Nombre_client,
    count(distinct t3.orderNumber) as Nombre_commande,
    sum(quantityOrdered*priceEach) as montant_total
            from Employees as t1
            left join Customers as t2 on t1.employeeNumber=t2.salesRepEmployeeNumber
            left join Orders as t3 on t2.customerNumber=t3.customerNumber
            left join OrderDetails as t4 on t3.orderNumber=t4.orderNumber
            group by t1.employeeNumber""",conn)
print(Q2)

Q3= pandas.read_sql_query("""SELECT t0.city, t0.country,
    Count(distinct t2.customerNumber) as Nombre_client,
    count(distinct t3.orderNumber) as Nombre_commande,
    sum(quantityOrdered*priceEach) as montant_total,
    count(distinct Case when t0.country !=t2.country then t2.customerNumber end) as Nombre_client_etranger
            from Offices t0
            left join Employees as t1 on t0.officeCode=t1.officeCode
            left join Customers as t2 on t1.employeeNumber=t2.salesRepEmployeeNumber
            left join Orders as t3 on t2.customerNumber=t3.customerNumber
            left join OrderDetails as t4 on t3.orderNumber=t4.orderNumber
            group by t1.officeCode""",conn)
print(Q3)

Q4= pandas.read_sql_query("""SELECT t0.productCode, t0.productName,
    Count(distinct t2.customerNumber) as Nombre_client,
    count(distinct t1.orderNumber) as Nombre_commande,
    sum(quantityOrdered) as quatite_total
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            left join Orders as t2 on t1.orderNumber=t2.orderNumber
            group by t0.productCode""",conn)
print(Q4)

Q5= pandas.read_sql_query("""SELECT t0.country,
    count(distinct t1.orderNumber) as nombre_commande,
    sum(quantityOrdered*priceEach) as montant_total,
    sum(amount) as montant_total_paye
            from Customers t0
            left join Orders as t1 on t0.customerNumber=t1.customerNumber
            left join OrderDetails as t2 on t1.orderNumber=t2.orderNumber
            left join Payments as t3 on t0.customerNumber=t3.customerNumber
            group by t0.country""",conn)
print(Q5)

Q6= pandas.read_sql_query("""SELECT t0.productLine,
                          t3.country,
    count(distinct t1.orderNumber) as nombre_commande
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            left join Orders as t2 on t1.orderNumber=t2.orderNumber
            left join Customers as t3 on t2.customerNumber=t3.customerNumber
            group by t0.productLine, t3.country""",conn)
print(Q6)

Q7= pandas.read_sql_query("""SELECT t0.productLine,
                          t3.country,
    sum(distinct t4.amount) as montant_total_paye
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            left join Orders as t2 on t1.orderNumber=t2.orderNumber
            left join Customers as t3 on t2.customerNumber=t3.customerNumber
            left join Payments as t4 on t3.customerNumber=t4.customerNumber
            group by t0.productLine, t3.country""",conn)
print(Q7)

Q8= pandas.read_sql_query("""SELECT t0.productName,
    avg(priceEach - buyPrice) as marge_moy
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            group by t0.productCode
            order by marge_moy desc
            limit 10""",conn)
print(Q8)

Q9= pandas.read_sql_query("""SELECT t0.productName,
                          t3.customerName,t3.customerNumber,
    (priceEach - buyPrice) as marge,
    sum(case when (priceEach - buyPrice) <=0 then 1 else 0 end) as nombre_commande
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            left join Orders as t2 on t1.orderNumber=t2.orderNumber
            left join Customers as t3 on t2.customerNumber=t3.customerNumber
            where marge <=0
            group by t0.productName""",conn)
print(Q9)

Q10= pandas.read_sql_query("""SELECT t0.customerName,
    sum(quantityOrdered*priceEach) as montant_total,
    sum(amount) as montant_total_paye
            from Customers t0
            left join Orders as t1 on t0.customerNumber=t1.customerNumber
            left join OrderDetails as t2 on t1.orderNumber=t2.orderNumber
            left join Products as ta on ta.productCode=t2.productCode
            left join Payments as t3 on t0.customerNumber=t3.customerNumber
            group by ta.productCode
            having montant_total_paye > montant_total""",conn)
print(Q10)


In [39]:

employe = pandas.read_sql_query(
    "SELECT * FROM Employees INNER JOIN Offices USING (officeCode);", 
    conn
)

In [40]:
employe = [
    employe.query('employeeNumber== @id')
    .drop(columns=["employeeNumber","officeCode"])
    .to_dict(orient = "records")
    for id in employe.employeeNumber
]
print(employe)

In [41]:
customers = pandas.read_sql_query(
    "SELECT * FROM Customers;", 
    conn
)


Payments = pandas.read_sql_query(
    "SELECT * FROM Payments;", 
    conn
)

In [42]:
Payments = [
    Payments.query('customerNumber== @id')
    .drop(columns=["customerNumber"])
    .to_dict(orient = "records")
    for id in customers.customerNumber
]

customers = customers.assign(payments = Payments)
customers.head()

customers = [
    customers.query('customerNumber== @id')
    .to_dict(orient = "records")
    for id in customers.customerNumber
]
print(customers)

In [47]:
order_detail = pandas.read_sql_query(
    "SELECT * FROM OrderDetails INNER JOIN Products USING (productCode);", 
    conn
)

order = pandas.read_sql_query(
    "SELECT * FROM Orders;", 
    conn
)

In [50]:
detail = [
    order_detail.query('orderNumber== @id')
    .drop(columns=["orderNumber","productCode"])
    .to_dict(orient = "records")
    for id in order.orderNumber
]

order = order.assign(details = detail)

order = [
    order.query('orderNumber== @id')
    .to_dict(orient = "records")
    for id in order.orderNumber
]

print(order)

[[{'orderNumber': 10100, 'orderDate': '2003/1/6 0:00:00', 'requiredDate': '2003/1/13 0:00:00', 'shippedDate': '2003/1/10 0:00:00', 'status': 'Shipped', 'comments': 'NULL', 'customerNumber': 363, 'details': [{'quantityOrdered': 30, 'priceEach': 171.7, 'orderLineNumber': 3, 'productName': '1917 Grand Touring Sedan', 'productLine': 'Vintage Cars', 'productScale': '4680.0', 'productVendor': 'Welly Diecast Productions', 'productDescription': 'This 1:18 scale replica of the 1917 Grand Touring car has all the features you would expect from museum quality reproductions: all four doors and bi-fold hood opening, detailed engine and instrument panel, chrome-look trim, and tufted upholstery, all topped off with a factory baked-enamel finish.', 'quantityInStock': 2724, 'buyPrice': 86.7, 'MSRP': 170.0}, {'quantityOrdered': 50, 'priceEach': 67.8, 'orderLineNumber': 2, 'productName': '1911 Ford Town Car', 'productLine': 'Vintage Cars', 'productScale': '4680.0', 'productVendor': 'Motor City Art Classic

In [ ]:
# Fermeture de la connexion : IMPORTANT à faire dans un cadre professionnel
conn.close()